<div style='background-color: #fff7f7; border: 3px solid Red;'>
    <font size="+1" color="Red">
        <b>Please look at our work!</b>
    </font>
</div>

<!-- </div>
<div style='background-color: #fff7f7; border: 3px solid yellow;'>
    <font size="+1" color="Red">
        <b><span style='color: green;'>Let's Support Each Other!!</span></b>
    </font>
</div> -->


<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>🗂️1. Install libraries and Download a pre-trained checkpoint 🗂️</b></font>
</div>


In [ ]:
# !pip install ultralytics

In [ ]:
# !kaggle datasets download sudipchakrabarty/kiit-mita

In [1]:
# !unzip kiit-mita.zip

In [ ]:
# import wandb
# !wandb.login(key="691f7f627317cf78520e137b1d29e92c725938aa") 

In [1]:
import os, cv2, yaml, random, numpy as np
from PIL import Image
from glob import glob
from matplotlib import pyplot as plt
from torchvision import transforms as T

In [2]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

 <div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>📈 2. Data Analysis 📈</b></font>
</div>


In [ ]:
def debug_directory(root):
    print(f"1. Checking if root exists: {os.path.exists(root)}")

    # Check YAML file
    yaml_path = os.path.join(root, "KIIT-MiTA.yml")
    print(f"\n2. Checking YAML file:")
    print(f"YAML path exists: {os.path.exists(yaml_path)}")
    if os.path.exists(yaml_path):
        with open(yaml_path, "r") as file:
            data = yaml.safe_load(file)
            print(f"YAML contents (names): {data.get('names', 'No names found')}")

    # Check directories and files
    for data_type in ["train", "valid", "test"]:
        print(f"\n3. Checking {data_type} directory:")

        # Check images directory
        image_dir = os.path.join(root, data_type, "images")
        print(f"{data_type}/images exists: {os.path.exists(image_dir)}")
        if os.path.exists(image_dir):
            images = glob(os.path.join(image_dir, "*"))
            print(f"Number of images found: {len(images)}")
            if images:
                print(f"First image path: {images[0]}")

        # Check labels directory
        label_dir = os.path.join(root, data_type, "labels")
        print(f"{data_type}/labels exists: {os.path.exists(label_dir)}")
        if os.path.exists(label_dir):
            labels = glob(os.path.join(label_dir, "*.txt"))
            print(f"Number of labels found: {len(labels)}")
            if labels:
                print(f"First label path: {labels[0]}")
                # Try to read first label file
                try:
                    with open(labels[0], "r") as f:
                        first_line = f.readline().strip()
                        print(f"First line of first label: {first_line}")
                except Exception as e:
                    print(f"Error reading label file: {e}")


# Run debug function
root = r"KIIT-MiTA"  # or use "archive/KIIT-MiTA"
debug_directory(root)

In [ ]:
class Visualization:
    def __init__(self, root, data_types, n_ims, rows, cmap=None):
        self.n_ims = n_ims
        self.rows = rows
        self.cmap = cmap
        self.data_types = data_types
        self.colors = ["firebrick", "darkorange", "blueviolet"]

        # Convert backslashes to forward slashes for consistency
        self.root = os.path.normpath(root)
        print(f"Initialized with root path: {self.root}")
        self.get_cls_names()
        self.get_bboxes()

    def get_cls_names(self):
        yaml_path = os.path.join(self.root, "KIIT-MiTA.yml")
        print(f"Reading YAML from: {yaml_path}")

        with open(yaml_path, "r") as file:
            data = yaml.safe_load(file)

        # Extract class names
        self.class_dict = {index: name for index, name in enumerate(data["names"])}
        print(f"Loaded class dictionary: {self.class_dict}")

    def get_bboxes(self):
        self.vis_datas = {}
        self.analysis_datas = {}
        self.im_paths = {}

        for data_type in self.data_types:
            print(f"\nProcessing {data_type} dataset")
            all_bboxes = []
            all_analysis_datas = {}

            image_dir = os.path.join(self.root, data_type, "images")
            print(f"Looking for images in: {image_dir}")

            im_paths = glob(os.path.join(image_dir, "*"))
            print(f"Found {len(im_paths)} images")

            for idx, im_path in enumerate(im_paths):
                # Get corresponding label path
                base_name = os.path.basename(im_path)
                name_without_ext = os.path.splitext(base_name)[0]
                lbl_path = os.path.join(
                    self.root, data_type, "labels", f"{name_without_ext}.txt"
                )

                if not os.path.isfile(lbl_path):
                    print(f"Warning: No label file for {im_path}")
                    continue

                bboxes = []
                with open(lbl_path, "r") as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) >= 5:  # Ensure we have class + 4 bbox coordinates
                            try:
                                class_idx = int(parts[0])
                                cls_name = self.class_dict[class_idx]
                                bbox = [cls_name] + [float(x) for x in parts[1:5]]
                                bboxes.append(bbox)

                                # Update analysis data
                                if cls_name not in all_analysis_datas:
                                    all_analysis_datas[cls_name] = 1
                                else:
                                    all_analysis_datas[cls_name] += 1

                            except (ValueError, IndexError, KeyError) as e:
                                print(f"Error processing line in {lbl_path}: {e}")
                                continue

                all_bboxes.append(bboxes)

            print(f"Processed {len(all_bboxes)} valid image-label pairs")
            print(f"Class distribution: {all_analysis_datas}")

            self.vis_datas[data_type] = all_bboxes
            self.analysis_datas[data_type] = all_analysis_datas
            self.im_paths[data_type] = im_paths

    def plot(self, rows, cols, count, im_path, bboxes):
        plt.subplot(rows, cols, count)

        try:
            or_im = np.array(Image.open(im_path).convert("RGB"))
            height, width, _ = or_im.shape

            for bbox in bboxes:
                class_id, x_center, y_center, w, h = bbox

                # Convert YOLO format to pixel values
                x_min = int((x_center - w / 2) * width)
                y_min = int((y_center - h / 2) * height)
                x_max = int((x_center + w / 2) * width)
                y_max = int((y_center + h / 2) * height)

                color = (
                    random.randint(0, 255),
                    random.randint(0, 255),
                    random.randint(0, 255),
                )
                cv2.rectangle(
                    img=or_im,
                    pt1=(x_min, y_min),
                    pt2=(x_max, y_max),
                    color=color,
                    thickness=3,
                )

            plt.imshow(or_im)
            plt.axis("off")
            plt.title(f"There is (are) {len(bboxes)} object(s) in the image.")

        except Exception as e:
            print(f"Error plotting image {im_path}: {e}")

        return count + 1

    def vis(self, save_name):
        print(f"\n{save_name.upper()} Data Visualization is in process...")

        if not self.vis_datas.get(save_name):
            print(f"No data found for {save_name}")
            return

        cols = max(1, self.n_ims // self.rows)
        count = 1

        plt.figure(figsize=(25, 20))

        data_len = len(self.vis_datas[save_name])
        indices = [
            random.randint(0, data_len - 1) for _ in range(min(self.n_ims, data_len))
        ]

        for idx, index in enumerate(indices):
            if count == self.n_ims + 1:
                break

            im_path = self.im_paths[save_name][index]
            bboxes = self.vis_datas[save_name][index]

            count = self.plot(self.rows, cols, count, im_path=im_path, bboxes=bboxes)

        plt.tight_layout()
        plt.show()

    def data_analysis(self, save_name, color):
        print(f"\nData analysis for {save_name}...")

        if not self.analysis_datas.get(save_name):
            print(f"No analysis data for {save_name}")
            return

        cls_names = list(self.analysis_datas[save_name].keys())
        counts = list(self.analysis_datas[save_name].values())

        if not cls_names or not counts:
            print(f"No classes or counts found for {save_name}")
            return

        fig, ax = plt.subplots(figsize=(30, 10))
        width = 0.7
        indices = np.arange(len(counts))

        ax.bar(indices, counts, width, color=color)
        ax.set_xlabel("Class Names", color="black", fontsize=12)
        ax.set_ylabel("Data Counts", color="black", fontsize=12)
        ax.set_title(
            f"{save_name.upper()} Dataset Class Imbalance Analysis", fontsize=14
        )

        ax.set_xticks(indices)
        ax.set_xticklabels(cls_names, rotation=45, ha="right")

        # Add value labels on top of bars
        for i, v in enumerate(counts):
            ax.text(i, v + max(counts) * 0.01, str(v), ha="center", va="bottom")

        plt.tight_layout()
        plt.show()

    def visualization(self):
        for save_name in self.data_types:
            self.vis(save_name)

    def analysis(self):
        for save_name, color in zip(self.data_types, self.colors):
            self.data_analysis(save_name, color)


# Usage
root = "KIIT-MiTA"  # Use forward slashes
vis = Visualization(
    root=root, data_types=["train", "valid", "test"], n_ims=20, rows=5, cmap="rgb"
)
vis.analysis()

<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>📊 3. Data Visualization 📊</b></font>
</div>


In [ ]:
vis.visualization()

<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>✨ 4. AI Model Train✨</b></font>
</div>


In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

In [ ]:
import yaml
import wandb
import mlflow
from mlflow import log_params, log_metric
from datetime import datetime
from ultralytics import YOLO  # Ensure you're importing the YOLO model

# Define root directory (update as per your file structure)
root = "/home/nic/workspace/Mlops/MHD/KIIT-MiTA"

# Load YAML content into a Python dictionary
yml_file_path = f"{root}/KIIT-MiTA.yml"

with open(yml_file_path, "r") as file:
    data = yaml.safe_load(file)

# Update the paths
data["train"] = f"{root}/train/images"
data["val"] = f"{root}/valid/images"
data["test"] = f"{root}/test/images"

print("Updated YAML Config:", data)

# Save the updated YAML content to a file
output_path = "updated_config.yml"
with open(output_path, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

# Initialize W&B project
wandb.init(project="yolo-training", config={
    "epochs": 30,
    "imgsz": 480,
    "batch": 16,
    "workers": 8
})

# Start MLflow experiment
mlflow.set_tracking_uri("http://192.168.29.161:8087")  # Your MLflow tracking URI
experiment_name = "YOLO Training"
mlflow.set_experiment(experiment_name)

# Initialize the YOLO model
model = YOLO("yolov11n.pt")  # Replace with the correct YOLO version and weights

with mlflow.start_run(run_name=f"YOLO_Run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
    # Log parameters to MLflow
    mlflow.log_param("epochs", 30)
    mlflow.log_param("imgsz", 480)
    mlflow.log_param("batch", 16)
    mlflow.log_param("workers", 8)
    mlflow.log_param("data", "updated_config.yml")

    # Start training
    train_results = model.train(
        data="updated_config.yml",
        epochs=30,
        imgsz=480,
        device=[0],
        batch=16,
        workers=8,
    )

    # Log metrics to W&B and MLflow
    # Using the correct attribute from the train_results
    metrics = train_results.box.map  # Access the mAP metric
    print("Training metrics:", metrics)

    if isinstance(metrics, dict):
        for key, value in metrics.items():
            if isinstance(value, (int, float)):
                wandb.log({key: value})
                mlflow.log_metric(key, value)

    # Save the model artifacts
    mlflow.log_artifacts("runs")  # Assuming 'runs' is the directory where YOLO saves results

# Finish W&B run
wandb.finish()

In [ ]:
import yaml
import wandb
import mlflow
from mlflow import log_params, log_metric
from datetime import datetime
from ultralytics import YOLO  # Ensure you're importing the YOLO model
import optuna  # For hyperparameter optimization

# Define root directory (update as per your file structure)
root = "/home/nic/workspace/Mlops/MHD_1/KIIT-MiTA"

# Load YAML content into a Python dictionary
yml_file_path = f"{root}/KIIT-MiTA.yml"

with open(yml_file_path, "r") as file:
    data = yaml.safe_load(file)

# Update the paths
data["train"] = f"{root}/train/images"
data["val"] = f"{root}/valid/images"
data["test"] = f"{root}/test/images"

# Save the updated YAML content to a file
output_path = "updated_config.yml"
with open(output_path, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

# Initialize W&B project
wandb.init(project="yolo-hyperparameter-tuning")

# MLflow tracking setup
mlflow.set_tracking_uri("http://192.168.29.161:8087")  # Your MLflow tracking URI
experiment_name = "YOLO Hyperparameter Tuning"
mlflow.set_experiment(experiment_name)


# Objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    epochs = trial.suggest_int("epochs", 10, 50, step=10)
    imgsz = trial.suggest_categorical("imgsz", [320, 480, 640])
    batch = trial.suggest_int("batch", 8, 32, step=8)
    learning_rate = trial.suggest_loguniform("lr0", 1e-4, 1e-2)
    workers = trial.suggest_int("workers", 4, 16, step=4)

    # Initialize the YOLO model
    model = YOLO("yolo11n.pt")  # Replace with the correct YOLO version and weights

    with mlflow.start_run(run_name=f"YOLO_Run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Log parameters to MLflow
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("imgsz", imgsz)
        mlflow.log_param("batch", batch)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("workers", workers)
        mlflow.log_param("data", "updated_config.yml")

        # Start training
        train_results = model.train(
            data="updated_config.yml",
            epochs=epochs,
            imgsz=imgsz,
            batch=batch,
            workers=workers,
            device=[0],
            lr0=learning_rate
        )

        # Extract and log the evaluation metric (e.g., mAP)
        metrics = train_results.box.map  # Adjust based on YOLO's output
        print(f"Trial metrics: {metrics}")

        # Log metrics to MLflow
        if isinstance(metrics, dict):
            for key, value in metrics.items():
                if isinstance(value, (int, float)):
                    mlflow.log_metric(key, value)

        # Log W&B metrics
        wandb.log({"mAP": metrics})

    # Return the primary metric for optimization
    return metrics  # Adjust based on your metric of interest


# Run Optuna study
study = optuna.create_study(direction="maximize")  # Maximize mAP
study.optimize(objective, n_trials=20)

# Best trial results
best_trial = study.best_trial
print("Best trial:")
print(f"  Value: {best_trial.value}")
print(f"  Params: {best_trial.params}")

# Save study results
with open("optuna_study_results.yml", "w") as file:
    yaml.dump(study.best_params, file)

# Finish W&B run
wandb.finish()


<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>😍 5. Learning Curves 😍</b></font>
</div>


In [ ]:
Image.open(f"runs/detect/train/confusion_matrix.png")

In [ ]:
Image.open(f"runs/detect/train/results.png")

In [ ]:
Image.open(f"runs/detect/train/P_curve.png")

In [ ]:
Image.open(f"runs/detect/train/PR_curve.png")

In [ ]:
Image.open(f"runs/detect/train/val_batch0_pred.jpg")

<div style='background-color: #fff7f7; border: 2px solid '>
    <font size="+2" color="blue" ><b>✅6. Inference with Test Dataset✅</b></font>
</div>


In [1]:
# use only if you want to load the model
# from ultralytics import YOLO
# model = YOLO("/home/nic/workspace/Mlops/MHD/runs/detect/train/weights/best.pt") 

In [14]:
inference_results = model("KIIT-MiTA/test/images", device=0, verbose=False)

In [ ]:
def inference_vis(res, n_ims, rows):
    cols = n_ims // rows
    plt.figure(figsize=(20, 10))
    for idx, r in enumerate(res):
        if idx == n_ims:
            break
        plt.subplot(rows, cols, idx + 1)
        or_im_rgb = np.array(Image.open(r.path).convert("RGB"))
        if idx == n_ims:
            break
        for i in r:
            for bbox in i.boxes:
                box = bbox.xyxy[0]
                x1, y1, x2, y2 = box
                coord1, coord2 = (int(x1), int(y1)), (int(x2), int(y2))
                cv2.rectangle(or_im_rgb, coord1, coord2, color=(255, 0, 0), thickness=2)
        plt.imshow(or_im_rgb)
        plt.title(f"Image#{idx + 1}")
        plt.axis("off")


inference_vis(res=inference_results, n_ims=15, rows=3)

In [ ]:
# Evaluate the model's performance on the validation set
results = model.val()

In [ ]:
# Export the model to ONNX format
model = YOLO("/home/nic/workspace/Mlops/MHD/runs/detect/train/weights/best.pt")
success = model.export(format="onnx", dynamic=True)

In [ ]:
# for tensorrt
# from ultralytics import YOLO

model = YOLO("/home/nic/workspace/Mlops/MHD/runs/detect/train/weights/best.pt")
model.export(format="onnx", device=0)